<a href="https://colab.research.google.com/github/saiyanprince10/ai-projects/blob/dev/IAI_Sp25_HW2_ssuryaar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IAI Assignment 2
## Author: Suryaa Raman
## Andrew ID: ssuryaar

In [1]:
import numpy as np
import random
import math
from collections import defaultdict

In [13]:
# Class to represent the 4x4 Tic-Tac-Toe game
class TicTacToe4x4:
    def __init__(self, dim=4):
        self.dim = dim
        self.board = np.zeros((dim, dim), dtype=int)
        self.current_player = 1
    def reset(self):
        self.board = np.zeros((self.dim, self.dim), dtype=int)

    def available_moves(self):
        # Returns a list of all available moves (empty spaces)
        return [(i, j) for i in range(self.dim) for j in range(self.dim) if self.board[i, j] == 0]
        # return np.argwhere(self.board == 0) - this returns an ndarray, but the comment says we must return a list

    def make_move(self, move:int, player:int):
        # Places the current player's mark on the board
        i, j = move
        if self.board[i, j] == 0:
          self.board[i, j] = player
          return True
        return False
        # Switches turn to the other player
        self.current_player = 3 - self.current_player

    def undo_move(self, move):
        # Removes a move from the board'
        i, j = move
        if self.board[i, j] != 0:
          self.board[i, j] = 0


    def check_winner(self,player:int):
        # Checks if a player has won the game
        # Check rows, columns, and diagonals for a win
        for i in range(self.dim):
            if np.all(self.board[i, :] == player) or np.all(self.board[:, i] == player):
                return True

        if np.all(np.diag(self.board) == player) or np.all(np.diag(np.fliplr(self.board)) == player):
            return True
        return False
    def is_draw(self):
        # Checks if the game is a draw
        return np.all(self.board != 0)

    def display(self):
        print(self.board)

In [ ]:
# Monte Carlo Tree Search Node
# class MCTSNode:
    # def __init__(self, state, parent=None):

   #  def is_fully_expanded(self):

    # def best_child(self, exploration_weight=1.4):

In [ ]:
# Monte Carlo Tree Search Algorithm
# class MCTS:
   # def __init__(self, itermax=1000, exploration_weight=1.4):

   # def search(self, state):

   # def _select(self, node):
        # Selection phase - navigate down the tree

   # def _expand(self, node):
        # Expansion phase - add new children nodes

   # def _simulate(self, state):
        # Simulation phase - play a random game

   # def _backpropagate(self, node, result):
        # Backpropagation phase - update values up the tree

In [ ]:
# Function to simulate a game between MCTS and a random player
# def play_game():

## Both players play randomly

In [3]:
import random
def random_player(board, player):
    moves = board.available_moves()
    return random.choice(moves)

In [14]:
def play_random_game():
    board = TicTacToe4x4()
    current_player = 1
    while True:
        # get a random move
        move = random_player(board, current_player)
        # make the move
        board.make_move(move, current_player)
        # check if the current player is winning
        if board.check_winner(current_player):
            # if yes, end the game
            print('player {} winning!'.format(current_player))
            board.display()
            return current_player
            # break

        # if not, check if it's a draw
        if board.is_draw():
            # if yes, end the game
            print('draw!')
            board.display()
            return 0
            # break

        # switch between Player 1 and 2
        current_player = 3 - current_player

In [17]:
import matplotlib.pyplot as plt

# Play the game 100 times and keep track of the results
results = []
for _ in range(5):
    print('-' * 50)
    results.append(play_random_game())

--------------------------------------------------
draw!
[[2 2 1 2]
 [2 1 2 1]
 [1 1 2 2]
 [2 1 1 1]]
--------------------------------------------------
draw!
[[1 2 2 1]
 [1 2 2 1]
 [2 1 1 2]
 [1 2 2 1]]
--------------------------------------------------
player 1 winning!
[[1 2 2 2]
 [1 2 1 2]
 [0 1 2 2]
 [1 1 1 1]]
--------------------------------------------------
player 1 winning!
[[1 1 2 2]
 [1 0 1 2]
 [1 0 0 0]
 [1 2 2 0]]
--------------------------------------------------
player 1 winning!
[[0 0 1 1]
 [0 0 2 1]
 [2 2 2 1]
 [2 0 1 1]]
--------------------------------------------------
draw!
[[2 1 1 2]
 [1 2 2 1]
 [1 2 2 2]
 [1 2 1 1]]
--------------------------------------------------
draw!
[[2 1 1 2]
 [2 1 2 1]
 [1 2 1 2]
 [1 2 2 1]]
--------------------------------------------------
draw!
[[2 1 1 1]
 [2 1 2 2]
 [1 2 2 1]
 [1 2 2 1]]
--------------------------------------------------
draw!
[[2 1 1 1]
 [2 2 2 1]
 [1 2 2 2]
 [2 1 1 1]]
----------------------------------------------

In [ ]:
# Main execution - run multiple games to test MCTS performance